In [46]:
import subprocess
import random
import fastaparser
import os

from src.dna import *

We will need to modify a sequences using the tool, then compare it against the background.

In [47]:
random.seed(867530)

min = 8
max = 20

sequences = [random_dna(i) for i in range(min, max+1)]

sequences

['CTTTAACA',
 'CATCGGACC',
 'TGGGGGGGCG',
 'TCCGTGCTGTA',
 'CGGGGCTAGTTC',
 'CTACGCCAATGTA',
 'GGTAACCGTATGTT',
 'ACTCTACGCAGCGCA',
 'CAAACTCGTTGGAACA',
 'TGAACATTTCGGTTATA',
 'CGCGCGAAGAACTTGGCA',
 'TCGATGAATTACGGAAACG',
 'ACCTATCGATCCAACCTAGG']

In [48]:
filepath = "data/"
filename = "U00096.3.fasta"

for seq in sequences:

    fasta_obj = read_fasta(filepath+filename)
    fasta_header = fasta_obj["header"]
    fasta_seq = fasta_obj["sequence"]

    new_sequence = insert_sequence(seq, fasta_seq)

    seq_id = str(len(seq)) + "_" + seq

    new_filename = filepath + "modified/" + seq_id + "_" + filename

    with open(new_filename, "w") as fasta_file:
        fasta_seq_mod = fastaparser.FastaSequence(
            sequence = new_sequence,
            # id = seq_id,
            description = filename + " with " + seq + "introduced"
        )

In [49]:
f = read_fasta(filename="data/U00096.3.fasta")

type(f)

dict

In [50]:
mod_files = os.listdir("data/modified/")

mod_files

['8_CTTTAACA_U00096.3.fasta',
 '9_CATCGGACC_U00096.3.fasta',
 '10_TGGGGGGGCG_U00096.3.fasta',
 '11_TCCGTGCTGTA_U00096.3.fasta',
 '12_CGGGGCTAGTTC_U00096.3.fasta',
 '13_CTACGCCAATGTA_U00096.3.fasta',
 '14_GGTAACCGTATGTT_U00096.3.fasta',
 '15_ACTCTACGCAGCGCA_U00096.3.fasta',
 '16_CAAACTCGTTGGAACA_U00096.3.fasta',
 '17_TGAACATTTCGGTTATA_U00096.3.fasta',
 '18_CGCGCGAAGAACTTGGCA_U00096.3.fasta',
 '19_TCGATGAATTACGGAAACG_U00096.3.fasta',
 '20_ACCTATCGATCCAACCTAGG_U00096.3.fasta']

In [51]:
for f in mod_files:
    print(f)
    print(f.split("_")[1])

8_CTTTAACA_U00096.3.fasta
CTTTAACA
9_CATCGGACC_U00096.3.fasta
CATCGGACC
10_TGGGGGGGCG_U00096.3.fasta
TGGGGGGGCG
11_TCCGTGCTGTA_U00096.3.fasta
TCCGTGCTGTA
12_CGGGGCTAGTTC_U00096.3.fasta
CGGGGCTAGTTC
13_CTACGCCAATGTA_U00096.3.fasta
CTACGCCAATGTA
14_GGTAACCGTATGTT_U00096.3.fasta
GGTAACCGTATGTT
15_ACTCTACGCAGCGCA_U00096.3.fasta
ACTCTACGCAGCGCA
16_CAAACTCGTTGGAACA_U00096.3.fasta
CAAACTCGTTGGAACA
17_TGAACATTTCGGTTATA_U00096.3.fasta
TGAACATTTCGGTTATA
18_CGCGCGAAGAACTTGGCA_U00096.3.fasta
CGCGCGAAGAACTTGGCA
19_TCGATGAATTACGGAAACG_U00096.3.fasta
TCGATGAATTACGGAAACG
20_ACCTATCGATCCAACCTAGG_U00096.3.fasta
ACCTATCGATCCAACCTAGG


In [52]:
# example motifs
# findMotifs.pl targets.fa fasta motifResults/ -fasta background.fa
for f in mod_files:

    subprocess.call([
        "findMotifs.pl",
        "data/U00096.3.fasta",
        "fasta",
        "homer-results/motifs_" + f.split("_")[1],
        "-fasta",
        "data/modified/" + f
    ])

FileNotFoundError: [Errno 2] No such file or directory: 'findMotifs.pl'